In [320]:
!pip install nltk

In [321]:
# импортируем библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# импортируем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split


# Регулярные выражения
import re

# импортируем другие необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import category_encoders as ce # импорт для работы с кодировщиком # кодирование OneHot
from sklearn import utils
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_regression, chi2

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer# оценка слов на позитивное или негативое настроение

In [322]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sf-booking/hotels_test.csv
/kaggle/input/sf-booking/hotels_train.csv
/kaggle/input/sf-booking/submission.csv


In [323]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [324]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [325]:
# Подгрузим наши данные из соревнования

DATA_DIR = '/kaggle/input/sf-booking/'
df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') # датасет для обучения
df_test = pd.read_csv(DATA_DIR+'hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн

# Общая информация о данных  

* hotel_address — адрес отеля;
* review_date — дата, когда рецензент разместил соответствующий отзыв;
* average_score — средний балл отеля, рассчитанный на основе * последнего комментария за последний год;
* hotel_name — название отеля;
* reviewer_nationality — страна рецензента;
* negative_review — отрицательный отзыв, который рецензент дал отелю;
* review_total_negative_word_counts — общее количество слов в отрицательном отзыв;
* positive_review — положительный отзыв, который рецензент дал отелю;
* review_total_positive_word_counts — общее количество слов в положительном отзыве.
* reviewer_score — оценка, которую рецензент поставил отелю на основе своего опыта;
* total_number_of_reviews_reviewer_has_given — количество отзывов, которые рецензенты дали в прошлом;
* total_number_of_reviews — общее количество действительных отзывов об отеле;
* tags — теги, которые рецензент дал отелю;
* days_since_review — количество дней между датой проверки и датой очистки;
* additional_number_of_scoring — есть также некоторые гости, которые просто поставили оценку сервису, но не оставили отзыв. Это число указывает, сколько там действительных оценок без проверки.
* lat — географическая широта отеля;
* lng — географическая долгота отеля.

# 1. Исследование данных

In [326]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

In [327]:
df_train.head(2)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097


In [328]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128935 entries, 0 to 128934
Data columns (total 16 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               128935 non-null  object 
 1   additional_number_of_scoring                128935 non-null  int64  
 2   review_date                                 128935 non-null  object 
 3   average_score                               128935 non-null  float64
 4   hotel_name                                  128935 non-null  object 
 5   reviewer_nationality                        128935 non-null  object 
 6   negative_review                             128935 non-null  object 
 7   review_total_negative_word_counts           128935 non-null  int64  
 8   total_number_of_reviews                     128935 non-null  int64  
 9   positive_review                             128935 non-null  object 
 

In [329]:
df_test.head(2)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,tags,days_since_review,lat,lng
0,Via Senigallia 6 20161 Milan Italy,904,7/21/2017,8.1,Hotel Da Vinci,United Kingdom,Would have appreciated a shop in the hotel th...,52,16670,Hotel was great clean friendly staff free bre...,62,1,"[' Leisure trip ', ' Couple ', ' Double Room '...",13 days,45.533137,9.171102
1,Arlandaweg 10 Westpoort 1043 EW Amsterdam Neth...,612,12/12/2016,8.6,Urban Lodge Hotel,Belgium,No tissue paper box was present at the room,10,5018,No Positive,0,7,"[' Leisure trip ', ' Group ', ' Triple Room ',...",234 day,52.385649,4.834443


In [330]:
sample_submission.head(2)

,reviewer_score,id
0,1,488440
1,10,274649


In [331]:
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128935 entries, 0 to 128934
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype
---  ------          --------------   -----
 0   reviewer_score  128935 non-null  int64
 1   id              128935 non-null  int64
dtypes: int64(2)
memory usage: 2.0 MB


### Определение пропущенных значений

In [332]:
df_train.isna().sum()

hotel_address                                    0
additional_number_of_scoring                     0
review_date                                      0
average_score                                    0
hotel_name                                       0
reviewer_nationality                             0
negative_review                                  0
review_total_negative_word_counts                0
total_number_of_reviews                          0
positive_review                                  0
review_total_positive_word_counts                0
total_number_of_reviews_reviewer_has_given       0
reviewer_score                                   0
tags                                             0
days_since_review                                0
lat                                           2448
lng                                           2448
dtype: int64

Видим, что пропущены значения только в полях с координатами отеля  
Определим их

In [333]:
mask = (df_train['lat'].isna()) | (df_train['lng'].isna())
grouped_hotels = df_train[mask].groupby(['hotel_address', 'hotel_name']).size().reset_index(name='count')
display(grouped_hotels['hotel_name'])

0                      Mercure Paris Gare Montparnasse
1                         Holiday Inn Paris Montmartre
2               Maison Albar Hotel Paris Op ra Diamond
3                       NH Collection Barcelona Podium
4                            City Hotel Deutschmeister
5                                     Hotel Park Villa
6                  Fleming s Selection Hotel Wien City
7                            Cordial Theaterhotel Wien
8                                  Hotel Daniel Vienna
9                                         Roomz Vienna
10                         Renaissance Barcelona Hotel
11    Austria Trend Hotel Schloss Wilhelminenberg Wien
12                                       Hotel Advance
13        Derag Livinghotel Kaiser Franz Joseph Vienna
14                                  Hotel City Central
15                  Hotel Pension Baron am Schottentor
16                                       Hotel Atlanta
Name: hotel_name, dtype: object

Определим координаты для 17 отелей и заполним пропущенные строки

In [334]:
hotels_coord = {
    'Mercure Paris Gare Montparnasse':[48.839701, 2.323519],
    'Holiday Inn Paris Montmartre':[48.888860, 2.333190],
    'Maison Albar Hotel Paris Op ra Diamond':[48.875140, 2.323420],
    'NH Collection Barcelona Podium':[41.391430, 2.177890],
    'City Hotel Deutschmeister':[48.220856, 16.366642],
    'Hotel Park Villa':[48.233495, 16.345556],
    'Fleming s Selection Hotel Wien City':[48.209095, 16.354568],
    'Cordial Theaterhotel Wien':[48.209530, 16.351515],
    'Hotel Daniel Vienna':[48.188835, 16.383810],
    'Roomz Vienna':[48.22201, 16.39331],
    'Renaissance Barcelona Hotel':[41.392430, 2.167500],
    'Austria Trend Hotel Schloss Wilhelminenberg Wien':[48.219555, 16.285566],
    'Hotel Advance':[41.38322, 2.16295],
    'Derag Livinghotel Kaiser Franz Joseph Vienna':[48.245914, 16.341188],
    'Hotel City Central':[48.213560, 16.379923],
    'Hotel Pension Baron am Schottentor':[48.216705, 16.359820],
    'Hotel Atlanta':[48.220310, 16.355880]
}

# Применяем функцию для заполнения координат
df_train.loc[mask, ['lat', 'lng']] = df_train[mask].apply(lambda x: pd.Series(hotels_coord[x['hotel_name']], index=['lat', 'lng']), axis=1)
# Проверка на количество нулевых значений после выполнения функции
df_train.isna().sum()

hotel_address                                 0
additional_number_of_scoring                  0
review_date                                   0
average_score                                 0
hotel_name                                    0
reviewer_nationality                          0
negative_review                               0
review_total_negative_word_counts             0
total_number_of_reviews                       0
positive_review                               0
review_total_positive_word_counts             0
total_number_of_reviews_reviewer_has_given    0
reviewer_score                                0
tags                                          0
days_since_review                             0
lat                                           0
lng                                           0
dtype: int64

### Работа с выбросами и удаление дупликатов

Исследуем data frame тестовый(так как, возможно, будут удаляться строки, а строки можно удалять только в тренировочном)  
Для Работы с выбросами будем использовать следубщие методы:  
* Метод межквартильного размаха (IQR)
* Метод z_score

In [335]:
def outliers_iqr_mod(data: pd.DataFrame, feature: str, left: float = 1.5, right: float = 1.5) -> pd.DataFrame:
    """
    Очистка данных от выбросов методом IQR:
    1 - Определяется разница между нижним(Q1) и верхним (Q3) квартилями;
    2 - Очистка от выбросов, находящихся за пределами left*IQR от Q1 и right*IQR до Q3.
    
    Args:
        data (pd.DataFrame): Изначальные Данные(DataFrame).
        feature (str): Параметр DataFrame, для которого определяются выбросы.
        left (float, optional): Левая граница порога оценки. Defaults to 1.5.
        right (float, optional): Правая граница порога оценки. Defaults to 1.5.

    Returns:
        pd.DataFrame: Очищенный от выбросов DataFrame.
    """
    # IQR = Q3(0.75) - Q1 (0.25)
    x = data[feature]
    quantile_1, quantile_3 = x.quantile(0.25), x.quantile(0.75)
    iqr = quantile_3 - quantile_1
    
    # Опредление границ выбросов и очистка данных 
    lower_bound = quantile_1 - (left*iqr)
    upper_bound = quantile_3 + (right*iqr)
    cleared_data = data[(x >= lower_bound) & (x <= upper_bound)]
    
    return cleared_data


def outliers_z_score_mode(data: pd.DataFrame, feature:str, log_scale=False, left:float = 3, right:float = 3) -> pd.DataFrame:
    """
    Определение выбросов с помощью метода z_score, чаще используется для нормального распределения, 
    так как показывает, насколько значение отклоняется от среднего значения в единицах среднего отклонения (std).

    Args:
        data (pd.DataFrame): Изначальные Данные(DataFrame).
        feature (str): Параметр DataFrame, для которого определяются выбросы.
        log_scale (bool, optional): Используется ли логарифмическая шкала при расчетах. Defaults to False.
        left (float, optional): Левый порог для расчета Z-score. Defaults to 3.
        right (float, optional): Правый порог для расчета Z-score. Defaults to 3.

    Returns:
        pd.DataFrame: Очищенный от выбросов DataFrame.
    """
    # Используется ли логарифмическая шкала
    if log_scale:
        x = np.log(data[feature] + 1)
    else:
        x = data[feature]
    
    # Расчет среднего и стандартного отклонения
    mu = x.mean()
    sigma = x.std()
    
    # Определение границ и очистка данных
    lower_bound = mu - (left*sigma)
    upper_bound = mu + (right*sigma)    
    cleared_data = data[(x >= lower_bound) & (x <= upper_bound)]
    
    return cleared_data

# Параметры для очистки с помощью Z-score
feature_for_z_score_log = ['additional_number_of_scoring', 'additional_number_of_scoring', 'review_total_positive_word_counts', 'review_total_negative_word_counts', 'total_number_of_reviews_reviewer_has_given']
for feature in feature_for_z_score_log:
    df_train = outliers_z_score_mode(df_train, feature, log_scale=True)

# Параметр для очистки с помощью IQR
column_for_iqr = 'average_score'
df_train = outliers_iqr_mod(df_train, column_for_iqr)


### Удаление дубликатов

In [336]:
df_train.drop_duplicates(inplace=True)

### Объединение обучающих и тестовых данных

In [337]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

Копируем данные, с которыми будм проводить дальнейшие манипуляции

In [338]:
hotels = data.copy()

In [339]:
hotels.isna().sum()

hotel_address                                   0
additional_number_of_scoring                    0
review_date                                     0
average_score                                   0
hotel_name                                      0
reviewer_nationality                            0
negative_review                                 0
review_total_negative_word_counts               0
total_number_of_reviews                         0
positive_review                                 0
review_total_positive_word_counts               0
total_number_of_reviews_reviewer_has_given      0
tags                                            0
days_since_review                               0
lat                                           820
lng                                           820
sample                                          0
reviewer_score                                  0
dtype: int64

Видим, что пропущены значения только в полях с координатами отеля  
Определим их

In [340]:
mask = (hotels['lat'].isna()) | (hotels['lng'].isna())
grouped_hotels = hotels[mask].groupby(['hotel_address', 'hotel_name']).size().reset_index(name='count')
display(grouped_hotels['hotel_name'])

0                      Mercure Paris Gare Montparnasse
1                         Holiday Inn Paris Montmartre
2               Maison Albar Hotel Paris Op ra Diamond
3                       NH Collection Barcelona Podium
4                            City Hotel Deutschmeister
5                                     Hotel Park Villa
6                  Fleming s Selection Hotel Wien City
7                            Cordial Theaterhotel Wien
8                                  Hotel Daniel Vienna
9                                         Roomz Vienna
10                         Renaissance Barcelona Hotel
11    Austria Trend Hotel Schloss Wilhelminenberg Wien
12                                       Hotel Advance
13        Derag Livinghotel Kaiser Franz Joseph Vienna
14                                  Hotel City Central
15                  Hotel Pension Baron am Schottentor
16                                       Hotel Atlanta
Name: hotel_name, dtype: object

Определим координаты для 17 отелей и заполним пропущенные строки

In [341]:
hotels_coord = {
    'Mercure Paris Gare Montparnasse':[48.839701, 2.323519],
    'Holiday Inn Paris Montmartre':[48.888860, 2.333190],
    'Maison Albar Hotel Paris Op ra Diamond':[48.875140, 2.323420],
    'NH Collection Barcelona Podium':[41.391430, 2.177890],
    'City Hotel Deutschmeister':[48.220856, 16.366642],
    'Hotel Park Villa':[48.233495, 16.345556],
    'Fleming s Selection Hotel Wien City':[48.209095, 16.354568],
    'Cordial Theaterhotel Wien':[48.209530, 16.351515],
    'Hotel Daniel Vienna':[48.188835, 16.383810],
    'Roomz Vienna':[48.22201, 16.39331],
    'Renaissance Barcelona Hotel':[41.392430, 2.167500],
    'Austria Trend Hotel Schloss Wilhelminenberg Wien':[48.219555, 16.285566],
    'Hotel Advance':[41.38322, 2.16295],
    'Derag Livinghotel Kaiser Franz Joseph Vienna':[48.245914, 16.341188],
    'Hotel City Central':[48.213560, 16.379923],
    'Hotel Pension Baron am Schottentor':[48.216705, 16.359820],
    'Hotel Atlanta':[48.220310, 16.355880]
}

# Применяем функцию для заполнения координат
hotels.loc[mask, ['lat', 'lng']] = hotels[mask].apply(lambda x: pd.Series(hotels_coord[x['hotel_name']], index=['lat', 'lng']), axis=1)
# Проверка на количество нулевых значений после выполнения функции
hotels.isna().sum()

hotel_address                                 0
additional_number_of_scoring                  0
review_date                                   0
average_score                                 0
hotel_name                                    0
reviewer_nationality                          0
negative_review                               0
review_total_negative_word_counts             0
total_number_of_reviews                       0
positive_review                               0
review_total_positive_word_counts             0
total_number_of_reviews_reviewer_has_given    0
tags                                          0
days_since_review                             0
lat                                           0
lng                                           0
sample                                        0
reviewer_score                                0
dtype: int64

### Удаление дубликатов

In [342]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505555 entries, 0 to 505554
Data columns (total 18 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               505555 non-null  object 
 1   additional_number_of_scoring                505555 non-null  int64  
 2   review_date                                 505555 non-null  object 
 3   average_score                               505555 non-null  float64
 4   hotel_name                                  505555 non-null  object 
 5   reviewer_nationality                        505555 non-null  object 
 6   negative_review                             505555 non-null  object 
 7   review_total_negative_word_counts           505555 non-null  int64  
 8   total_number_of_reviews                     505555 non-null  int64  
 9   positive_review                             505555 non-null  object 
 

In [343]:
data.nunique()

hotel_address                                   1493
additional_number_of_scoring                     480
review_date                                      731
average_score                                     34
hotel_name                                      1492
reviewer_nationality                             226
negative_review                               322466
review_total_negative_word_counts                402
total_number_of_reviews                         1142
positive_review                               405661
review_total_positive_word_counts                314
total_number_of_reviews_reviewer_has_given       158
tags                                           54718
days_since_review                                731
lat                                             1489
lng                                             1489
sample                                             2
reviewer_score                                    38
dtype: int64

In [344]:
data.head(2)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,tags,days_since_review,lat,lng,sample,reviewer_score
0,Via Senigallia 6 20161 Milan Italy,904,7/21/2017,8.1,Hotel Da Vinci,United Kingdom,Would have appreciated a shop in the hotel th...,52,16670,Hotel was great clean friendly staff free bre...,62,1,"[' Leisure trip ', ' Couple ', ' Double Room '...",13 days,45.533137,9.171102,0,0.0
1,Arlandaweg 10 Westpoort 1043 EW Amsterdam Neth...,612,12/12/2016,8.6,Urban Lodge Hotel,Belgium,No tissue paper box was present at the room,10,5018,No Positive,0,7,"[' Leisure trip ', ' Group ', ' Triple Room ',...",234 day,52.385649,4.834443,0,0.0


In [345]:
data.describe()

,additional_number_of_scoring,average_score,review_total_negative_word_counts,total_number_of_reviews,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,lat,lng,sample,reviewer_score
count,505555.000000,505555.000000,505555.000000,505555.000000,505555.000000,505555.000000,504735.000000,504735.000000,505555.000000,505555.000000
mean,501.589522,8.419481,18.378546,2754.511824,17.741994,6.911309,49.417364,2.891116,0.744963,6.279731
std,503.803284,0.518166,29.474036,2328.912620,21.186061,9.691692,3.475639,4.660392,0.435882,3.928847
min,1.000000,5.200000,0.000000,43.000000,0.000000,1.000000,41.328376,-0.369758,0.000000,0.000000
25%,169.000000,8.100000,2.000000,1161.000000,5.000000,1.000000,48.213560,-0.142613,0.000000,0.000000
50%,341.000000,8.400000,9.000000,2130.000000,11.000000,3.000000,51.499968,0.019886,1.000000,7.900000
75%,681.000000,8.800000,23.000000,3636.000000,22.000000,8.000000,51.516191,4.834975,1.000000,9.600000
max,2682.000000,9.800000,408.000000,16670.000000,383.000000,315.000000,52.400181,16.429233,1.000000,10.000000


In [346]:
data.describe(include='object')

,hotel_address,review_date,hotel_name,reviewer_nationality,negative_review,positive_review,tags,days_since_review
count,505555,505555,505555,505555,505555,505555,505555,505555
unique,1493,731,1492,226,322466,405661,54718,731
top,163 Marsh Wall Docklands Tower Hamlets London ...,8/2/2017,Britannia International Hotel Canary Wharf,United Kingdom,No Negative,No Positive,"[' Leisure trip ', ' Couple ', ' Standard Doub...",1 days
freq,4785,2523,4785,240336,126310,34923,4861,2523


# 2.Формирование новых признаков 

Копирование информации перед обработкой

### Определение дня/время года заселения

In [347]:
# Преобразования в date_time
hotels['review_date'] = pd.to_datetime(hotels['review_date'], format='%m/%d/%Y')
# Выделение времени года и дня заселения
hotels['review_quarter'] = hotels['review_date'].dt.quarter
hotels['review_day_of_week'] = hotels['review_date'].dt.day_of_week

### Обработка информации tags

In [348]:
def split_tags(tag:str) -> pd.Series:
    """
    Обработка параметра tags и декодирование зашитой в нем информации
    
    Паттерн для этого параметра:
    ['Путешествие с животным', 'тип путешествия', 'Вид компании', 'Тип номера', 'Продолжительность остановки', 'С какого устройства был оставлен отзыв']

    Args:
        tag (str): Строка с перечисленными тегами.

    Returns:
        pd.Series: Данные после обработки функции
        
    Пример запуска:
    tag = "[' Leisure trip ', ' Couple ', ' King Room ', ' Stayed 1 night ']"
    split_tags(tag)
    
    Результат:
     - Without pet
     - Leisure trip
     - Couple
     - King Room
     - 1
     - Submitted Not from a mobile device
    """
    
    pattern = re.compile(
    r"\[(?:' (With a pet) '(?:, )?)?"  # с животным или без
    r"(?:' (.*? trip) '(?:, )?)?"  # Тип поездки
    r"(?:' (Solo.*?|Travelers with friends|Couple|Group.*?|Family.*?) ',?\s*)?"  # Состав группы
    r"(?:' (?!Stayed|Submitted)(.*?) ',?\s*)?"  # Тип номера
    r"(?:' Stayed (\d+) night[s]? ?,?\s*')?"  # Продолжительность
    r"(?:,\s*' (Submitted from a mobile device) ')?\]"  # Дополнительная информация
    )
    # Поиск закономерностей
    match = pattern.match(tag)
    if match:
        trip_with_pet = match.group(1) if match.group(1) else 'Without a pet'
        trip_type = match.group(2) if match.group(2) else None
        group_type = match.group(3) if match.group(3) else None
        room_type = match.group(4) if match.group(4) else None
        stay_duration = int(match.group(5)) if match.group(5) else None
        submitted_type = match.group(6) if match.group(6) else 'Submitted Not from a mobile device'
    return pd.Series([trip_with_pet, trip_type, group_type, room_type, stay_duration, submitted_type])


hotels[['trip_with_pet', 'trip_type', 'group_type', 'room_type', 'stay_duration', 'submitted_type']] = hotels['tags'].apply(split_tags)

Пропущенные значения в исправленном DataFrame

In [349]:
hotels.isna().sum()

hotel_address                                     0
additional_number_of_scoring                      0
review_date                                       0
average_score                                     0
hotel_name                                        0
reviewer_nationality                              0
negative_review                                   0
review_total_negative_word_counts                 0
total_number_of_reviews                           0
positive_review                                   0
review_total_positive_word_counts                 0
total_number_of_reviews_reviewer_has_given        0
tags                                              0
days_since_review                                 0
lat                                               0
lng                                               0
sample                                            0
reviewer_score                                    0
review_quarter                                    0
review_day_o

Заполняем пропущенные значения  
Для типа номера пропущенные значения не заполняются, ввиду последующей обработки

In [350]:
most_common_trip_type = hotels['trip_type'].mode()[0]
# Заменяем значения None на наиболее часто встречающееся значение
hotels.loc[hotels['trip_type'].isna(), 'trip_type'] = most_common_trip_type

most_common_stay_duration = hotels['stay_duration'].mode()[0]
# Заменяем значения None на наиболее часто встречающееся значение
hotels.loc[hotels['stay_duration'].isna(), 'stay_duration'] = most_common_stay_duration

#### Преобразования типа номера

In [351]:
# Заполнение NaN пустыми строками
hotels['room_type'] = hotels['room_type'].fillna(' ')

# Определение условий для категорий
conditions = [
    hotels['room_type'].str.contains('Single', case=False),
    hotels['room_type'].str.contains('Double|Standard Room', case=False),
    hotels['room_type'].str.contains('Suite', case=False),
    hotels['room_type'].str.contains('King', case=False),
    hotels['room_type'].str.contains('Queen', case=False),
    hotels['room_type'].str.contains(' ', case=False)
]

# Категории для выбора
choices = ['Single Room', 'Double Room', 'Suite', 'King Room', 'Queen Room', None]

# Применение np.select для создания нового столбца
hotels['room_type'] = np.select(conditions, choices, default='Other')

# Группировка по новым типам комнат
most_common_hotel_type = hotels['room_type'].mode()[0]
hotels['room_type'] = hotels['room_type'].fillna(most_common_hotel_type)

### Определение типа настроения оставляющего отзыв

In [352]:
sia = SentimentIntensityAnalyzer()

def analyze_review(review: str) -> float:
    # Анализ положительного отзыва
    if len(review['positive_review']) < 30:
        if any(substring in review['positive_review'].lower() for substring in ['all', 'everything', 'excellent', 'great', 'best']):
            positive_compound = 1
        elif any(substring in review['positive_review'].lower() for substring in ['no positive', 'n a', 'nothing', 'rude', 'bad', 'worst']):
            positive_compound = 0
        else:
            positive_compound = sia.polarity_scores(review['positive_review'])['compound']
    else:
        positive_compound = sia.polarity_scores(review['positive_review'])['compound']
    
    # Анализ отрицательного отзыва
    if len(review['negative_review']) < 30:
        if any(substring in review['negative_review'].lower() for substring in ['all', 'everything', 'worst']):
            negative_compound = -1
        elif any(substring in review['negative_review'].lower() for substring in ['no negative', 'n a', 'nothing', 'great', 'perfect', 'good', 'best']):
            negative_compound = 0
        else:
            negative_compound = sia.polarity_scores(review['negative_review'])['compound']
    else:
        negative_compound = sia.polarity_scores(review['negative_review'])['compound']
    
    # Рассчет среднего значения по compound
    mean_compound = (positive_compound + negative_compound) / 2
    return mean_compound

    
hotels['review_sentiments'] = hotels[['positive_review', 'negative_review']].apply(analyze_review, axis=1)

### Поиск страны, в которой находится отель

In [353]:
hotels['hotel_country'] = hotels['hotel_address'].apply(lambda address: ' '.join(address.split()[-2:]) if address.split()[-1] == 'Kingdom' else address.split()[-1])

### Формирование количества дней с отзыва

In [354]:
def get_days_since_review(tag):
    pattern = re.compile(r"\s*(\d*)\s*")                                                                   
    return int(pattern.match(tag).group(1))

hotels['days_since_review'] = hotels['days_since_review'].apply(get_days_since_review)

### После преобразования новых признаков, удалим ненужные

In [355]:
hotels_droped = hotels.copy()

columns_for_delete = ['hotel_address', 'hotel_name', 'negative_review', 'positive_review', 
                      'tags', 'review_date', 'lat', 'lng', 'trip_with_pet',
                      'room_type', 'group_type', 'days_since_review', 'additional_number_of_scoring']
hotels_droped.drop(columns_for_delete, axis=1, inplace=True)

# 3.Номализация и стандартизация

In [356]:
# Применение стандартизации к ненормально распределенным параметрам
standart_scaler_features = ['review_total_negative_word_counts', 'total_number_of_reviews', 'review_total_positive_word_counts', 'total_number_of_reviews_reviewer_has_given']
standart_scaler = StandardScaler()
hotels_droped[standart_scaler_features] = standart_scaler.fit_transform(hotels_droped[standart_scaler_features])

# Применение нормализации к нормально распределенным параметрам
min_max_scaler_features = ['average_score']
min_max_scaler = MinMaxScaler()
hotels_droped[min_max_scaler_features] = min_max_scaler.fit_transform(hotels_droped[min_max_scaler_features])

In [359]:
hotels_droped.columns

Index(['average_score', 'reviewer_nationality',
       'review_total_negative_word_counts', 'total_number_of_reviews',
       'review_total_positive_word_counts',
       'total_number_of_reviews_reviewer_has_given', 'sample',
       'reviewer_score', 'review_quarter', 'review_day_of_week', 'trip_type',
       'stay_duration', 'submitted_type', 'review_sentiments',
       'hotel_country'],
      dtype='object')

# 4. Кодирование признаков

Рассмотрим первый вариант без кодирования признаков

In [360]:
# Копирование исходного DataFrame перед кодированием
#hotels_encoded = hotels.copy()

# Списки параметров для разных типоа кодирования
one_hot_features = []
binary_features = []

for column in hotels_droped.columns:
    # Кодируем только тип данных - object
    if hotels_droped[column].dtype == 'object':
        # Если в признаке более 15 уникальных значений - применяем Binary
        # Иначе - One Hot Encoding
        unique_values = hotels_droped[column].nunique()
        if unique_values < 15:
            one_hot_features.append(column)
        else:
            binary_features.append(column)

# Кодирование признаков
# One-hot encoding
hotels_encoded = pd.get_dummies(hotels_droped, columns=one_hot_features, drop_first=True)
# Binary encoding
binary_encoder = ce.BinaryEncoder(cols=binary_features)
encoded = binary_encoder.fit_transform(hotels_droped[binary_features])
# объединение данных
hotels_encoded = pd.concat([hotels_encoded, encoded], axis=1)
hotels_encoded.drop(binary_features, axis=1, inplace=True)

In [ ]:
#hotels_encoded.duplicated().sum()

In [ ]:
#hotels_encoded['sample'].value_counts(normalize=True)

# 5. Обучение модели

In [361]:
hotels_encoded.columns

Index(['average_score', 'review_total_negative_word_counts',
       'total_number_of_reviews', 'review_total_positive_word_counts',
       'total_number_of_reviews_reviewer_has_given', 'sample',
       'reviewer_score', 'review_quarter', 'review_day_of_week',
       'stay_duration', 'review_sentiments', 'trip_type_Leisure trip',
       'submitted_type_Submitted from a mobile device', 'hotel_country_France',
       'hotel_country_Italy', 'hotel_country_Netherlands',
       'hotel_country_Spain', 'hotel_country_United Kingdom',
       'reviewer_nationality_0', 'reviewer_nationality_1',
       'reviewer_nationality_2', 'reviewer_nationality_3',
       'reviewer_nationality_4', 'reviewer_nationality_5',
       'reviewer_nationality_6', 'reviewer_nationality_7'],
      dtype='object')

Выделим обучающую и тестовую части

In [366]:
# Разделение данных на тренировочные и тестовые
train_data = hotels_encoded.query('sample == 1').drop(['sample'], axis=1)
test_data = hotels_encoded.query('sample == 0').drop(['sample'], axis=1)

# Целевая переменная и признаки
y = train_data.reviewer_score.values  # таргет
X = train_data.drop(['reviewer_score'], axis=1)  # признаки

# Определение числовых и категориальных признаков
num_cols = ['review_total_negative_word_counts', 'total_number_of_reviews', 
            'review_total_positive_word_counts', 'total_number_of_reviews_reviewer_has_given',
            'stay_duration', 'review_sentiments']

cat_cols = ['average_score', 'submitted_type_Submitted from a mobile device', 'hotel_country_France', 
            'hotel_country_Italy', 'hotel_country_Netherlands', 'hotel_country_Spain', 
            'hotel_country_United Kingdom', 'reviewer_nationality_0', 'reviewer_nationality_1', 
            'reviewer_nationality_2', 'reviewer_nationality_3', 'reviewer_nationality_4', 
            'reviewer_nationality_5', 'reviewer_nationality_6', 'reviewer_nationality_7']

# Разделяем данные на числовые и категориальные признаки
X_numeric = X[num_cols]
X_categorical = X[cat_cols]

# Применяем SelectKBest на тренировочных данных
selector_numeric = SelectKBest(f_classif, k=5)
X_numeric_selected = selector_numeric.fit_transform(X_numeric, y)

selector_categorical = SelectKBest(mutual_info_regression, k=5)
X_categorical_selected = selector_categorical.fit_transform(X_categorical, y)

# Объединяем выбранные числовые и категориальные признаки
X_selected = np.hstack([X_numeric_selected, X_categorical_selected])

# Разделение данных на тренировочные и тестовые
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=RANDOM_SEED)

# Проверяем размеры данных
print(f"Test data shape: {test_data.shape}, Train data shape: {train_data.shape}")
print(f"X shape: {X.shape}, X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")

# Подготовка тестовых данных
X_test_data_numeric = test_data[num_cols]
X_test_data_categorical = test_data[cat_cols]

# Применяем тот же отбор признаков на тестовых данных
X_test_data_numeric_selected = selector_numeric.transform(X_test_data_numeric)
X_test_data_categorical_selected = selector_categorical.transform(X_test_data_categorical)

# Объединяем выбранные признаки тестового набора
X_test_selected = np.hstack([X_test_data_numeric_selected, X_test_data_categorical_selected])

# Создание модели RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

# Обучение модели
model.fit(X_train, y_train)

# Предсказание на тестовых данных
y_pred = model.predict(X_test)

print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

# Удаляем 'reviewer_score' из тестового набора данных (если оно есть)
test_data = test_data.drop(['reviewer_score'], axis=1, errors='ignore')

# Предсказание для тестовых данных
predict_submission = model.predict(X_test_selected)
# Записываем предсказания в файл для отправки
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)

# Выводим первые 10 строк
print(sample_submission.head(10))

Test data shape: (128935, 25), Train data shape: (376620, 25)
X shape: (376620, 24), X_train shape: (301296, 10), X_test shape: (75324, 10)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   27.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.0min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.4s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


MAPE: 0.12620832979441843


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    1.1s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    2.5s finished


   reviewer_score      id
0           8.407  488440
1           7.914  274649
2           8.696  374688
3           9.660  404352
4           9.524  451596
5           9.453  302161
6           8.503  317079
7           8.016   13963
8           8.502  159785
9           8.373  195089
